<a href="https://colab.research.google.com/github/kerryback/Classic_Tests/blob/main/4_Gibbons-Ross-Shanken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

We illustrate the Gibbons-Ross-Shanken (GRS) test.  We work with factors that are excess returns and test assets that are excess returns (an excess return is a long-minus-short return, where the short could be the risk-free asset).  The test is based on the contribution that the test assets make to the maximum squared Sharpe ratio achievable with the factors.  The maximum squared Sharpe ratio of any set of excess returns is $\mu'\Sigma^{-1}\mu$, where $\mu$ is the vector of means and $\Sigma$ is the covariance matrix.  By regressing the test assets on the factors, we can decompose them as factor risks plus what is left over, which is $\alpha+\varepsilon$ from the regression.  The factor risks do not contribute anything extra to the maximum squared Sharpe ratio of the factors, and the $\alpha+\varepsilon$ is orthogonal to the factors.  Consequently, it can be shown that the maximum squared Sharpe ratio achievable from the combined assets (test assets plus factors) is
$$\mu_C' \Sigma_C^{-1} \mu_C = \mu_F' \Sigma_F^{-1} \mu_F  + \alpha'\Sigma_\varepsilon^{-1}\alpha $$
where $F$ denotes the factors, $\alpha$ is the vector of alphas of the test assets, and $\Sigma_\varepsilon$ is the covariance matrix of the regression residuals.

The GRS statistic is
$$ \frac{T}{N} \cdot \frac{T-N-L}{T-L-1} \cdot \frac{\alpha'\Sigma_\varepsilon^{-1}\alpha}{1+\mu_F' \Sigma_F^{-1} \mu_F} = \frac{T}{N} \cdot \frac{T-N-L}{T-L-1} \cdot \frac{\mu_C' \Sigma_C^{-1} \mu_C - \mu_F' \Sigma_F^{-1} \mu_F}{1+\mu_F' \Sigma_F^{-1} \mu_F}$$
where $T$ is the length of the sample (number of time periods), $N$ is the number of test assets, and $L$ is the number of factors.  Under the null hypothesis that the factor model holds and under a normality assumption, the GRS statistic has an $F$ distribution with $N$ and $T-N-L$ degrees of freedom.

In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import statsmodels.api as sm
from scipy.stats import f

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Define the GRS test


In [ ]:
def grs(d,factors) :
    def sqSh(df) :
        return df.mean() @ np.linalg.solve(df.cov(),df.mean())
    C = sqSh(d)
    F = sqSh(d[factors])
    T = d.shape[0]
    L = len(factors)
    N = d.shape[1] - L
    stat = T*(T-N-L)*(C-F) / (N*(T-L-1)*(1+F))
    pval = 1 - f.cdf(stat,N,T-N-L)
    return stat, pval


# Read the file created and saved in the Black_Jensen_Scholes notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/crsp_compustat_example2.csv', parse_dates=['date'])
df.date = df.date.dt.to_period('M')
df = df.sort_values(by=['permno','date'])

Mounted at /content/drive


# Read the original Fama-French factors

In [ ]:
ff = pdr('F-F_Research_Data_Factors','famafrench',start='1962-01-01')[0] / 100

# Example 1: Test the CAPM with 10 beta-sorted portfolios

In [ ]:
# compute decile returns
df['decile'] = df.groupby('date').beta.apply(lambda x: pd.qcut(x,10,labels=range(1,11)))
rets = df.groupby(['date','decile']).apply(lambda d: (d.me*d.ret).sum() / d.me.sum())
rets = rets.unstack()
df = df.drop(columns=['decile'])

# add market excess return
rets['Mkt-RF'] = ff['Mkt-RF']
rets = rets.dropna()

# run the GRS test
grs(rets,['Mkt-RF'])


(14.6008014101939, 1.1102230246251565e-16)

# Example 2: Test the original FF model with 10 operating profit sorted portfolios

In [ ]:
# compute decile returns 
df2 = df.dropna(subset=['inv','me','ret'],how='any').copy()
df2['decile'] = df2.groupby('date').op.apply(lambda x: pd.qcut(x,10,labels=range(1,11)))
rets = df2.groupby(['date','decile']).apply(lambda d: (d.me*d.ret).sum() / d.me.sum())
rets = rets.unstack()

# add Fama-French factors
rets = rets.merge(ff[['Mkt-RF','SMB','HML']],left_index=True,right_index=True,how='inner')

# run the GRS test
grs(rets,['Mkt-RF','SMB','HML'])

(57.97244321488265, 1.1102230246251565e-16)